In [13]:
import pandas as pd
import requests
import time

def get_geocode_from_publisher(Affiliation):
    # Insert your actual Google Maps Geocoding API key
    GOOGLE_MAPS_API_KEY = ""
    GEOCODE_API_URL = "https://maps.googleapis.com/maps/api/geocode/json"

    parameters = {
        "address": Affiliation,
        "key": GOOGLE_MAPS_API_KEY
    }

    try:
        response = requests.get(GEOCODE_API_URL, params=parameters)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        print ("Oops: Something went wrong", err)
        return None, None

    data = response.json()

    # Printing out the response data to inspect
    print(data)

    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    else:
        return None, None

# Read the Excel file
# This file is the excel equivaltent to the CSV file 'Individual affiliation effects.csv' in the OSF repository 
df = pd.read_excel('/S2_Data_analyses/Individual affiliation effects.xlsx')

# Correct possible encoding issues if they exist
df['Affiliation'] = df['Affiliation'].str.encode('latin1').str.decode('utf8', errors='ignore')

# Initialize empty lists for lat and lon
lat_list = []
lon_list = []

# Loop through each row
for affiliation in df['Affiliation']:
    lat, lon = get_geocode_from_publisher(affiliation)
    lat_list.append(lat)
    lon_list.append(lon)
    time.sleep(1)  # sleep for 1 second to avoid rate limit

# Add latitude and longitude to dataframe
df['latitude'] = lat_list
df['longitude'] = lon_list

# Write the dataframe to a new CSV file
df.to_csv('/S2_Data_analyses/Affiliation_with_geocode.csv', index=False, sep=';')


{'results': [{'address_components': [{'long_name': '5', 'short_name': '5', 'types': ['street_number']}, {'long_name': 'Yi He Yuan Lu', 'short_name': 'Yi He Yuan Lu', 'types': ['route']}, {'long_name': 'Hai Dian Qu', 'short_name': 'Hai Dian Qu', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Bei Jing Shi', 'short_name': 'Bei Jing Shi', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'China', 'short_name': 'CN', 'types': ['country', 'political']}, {'long_name': '100871', 'short_name': '100871', 'types': ['postal_code']}], 'formatted_address': '5 Yi He Yuan Lu, Hai Dian Qu, Bei Jing Shi, China, 100871', 'geometry': {'location': {'lat': 39.986913, 'lng': 116.3058739}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 39.9883056302915, 'lng': 116.3070124802915}, 'southwest': {'lat': 39.9856076697085, 'lng': 116.3043145197085}}}, 'partial_match': True, 'place_id': 'ChIJdaICDUtR8DURywTKgdicwXs', 'plus_code': {'compound_code': 'X